# Residential Real Estate vs Supply, Inflation, and Interest Rate #

## Description ##
Our goal is to determine how the value of residential real estate relates to supply, inflation, and interest rate.  Additionally, we will compare real estate returns to the returns of the  S&P 500 and XLRE (Real Estate Select Sector).

## Objectives ##
We intend to identify ideal periods to transact residential real estate based on valuation, inflation, and interest rate.

In [ ]:
# Imports
import pandas as pd
import os
from dotenv import load_dotenv
from fredapi import Fred
import hvplot.pandas

# Load Environment Variables
load_dotenv()

# setup fred api client
FRED_API_KEY = os.getenv('FRED_API_KEY')
fred = Fred(api_key=FRED_API_KEY)

## Load all Fred Data

In [ ]:
dataframes = {}
FRED_SERIES_KEYS = ["FPCPITOTLZGUSA", "DFF", "MSACSR", "USSTHPI"]
for FRED_SERIES_KEY in FRED_SERIES_KEYS:
    series = fred.get_series(FRED_SERIES_KEY)
    frame = {
        "date": series.keys(),
        "value": series.values,
    }
    dataframes[FRED_SERIES_KEY] = pd.DataFrame(frame).set_index("date")
    dataframes[FRED_SERIES_KEY][f"pct_change {FRED_SERIES_KEY}"] = dataframes[FRED_SERIES_KEY].pct_change()
    dataframes[FRED_SERIES_KEY][f"diff {FRED_SERIES_KEY}"] = dataframes[FRED_SERIES_KEY]["value"].diff()
    dataframes[FRED_SERIES_KEY] = dataframes[FRED_SERIES_KEY].dropna()
    

## Residential Real Estate Value vs Inflation

In [ ]:
# Inflation, consumer prices for the United States
display(dataframes["FPCPITOTLZGUSA"]["value"].hvplot(title="Infaltion, consumer prices for the United States"))

# House price Index for the United States
display(dataframes["USSTHPI"]["value"].hvplot(title="All Transactions House Price Index for the United States"))

In [ ]:
# Rename Columns to 'Inflation in %', Inflation pct_change'
dataframes["FPCPITOTLZGUSA"].columns = ['Inflation','Inflation pct_change','Difference in Inflation'] 
display(dataframes["FPCPITOTLZGUSA"].head())

# Rename Columns to 'House Price Index', HPI pct_change'
dataframes["USSTHPI"].columns = ['HPI','HPI pct_change','Difference in HPI']
dataframes["USSTHPI"].head()

#### Combine Infaltion and Housing Price Index for the United States

In [ ]:
# Use 'concat' function to combine the two DataFrames by matching indexes.
inflation_combined_df = pd.concat([dataframes["FPCPITOTLZGUSA"], dataframes["USSTHPI"]],axis="columns", join="inner")
inflation_combined_df.head()

In [ ]:
# Drop columns 'Inflation in %' and 'House Price Index'
inflation_combined_df = inflation_combined_df.drop(columns=["Inflation","Inflation pct_change","HPI","HPI pct_change"])
inflation_combined_df.head()

#### Conduct House Price Index and Inflation Analysis

In [ ]:
# Plot Pct difference for combined dataframes
inflation_combined_df.hvplot(title="Housing Price Index, Inflation for the United States",xlabel="Date", ylabel="Difference")

## Residential Real Estate Value vs Federal Funds Effective Rate

In [ ]:
# Federal Funds Effective Rate
effective_rate_df = fred.get_series('DFF')
effective_rate_df.hvplot()

## Residential Real Estate Value vs Monthly Supply of New Houses

In [ ]:
# Monthly Supply of New Houses in the United States 
monthly_supply_df = fred.get_series('MSACSR')
monthly_supply_df.hvplot()

## Residential Real Estate vs S&P500

## Residential Real Estate vs XLRE